<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np
import pandas as pd
from collections import Counter

### Datos

In [2]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [3]:
# Función para armar el diccionario con los documentos y el vocabulario del corpus
def doc_voc(corpus):
    '''
    corpus: corpus que contiene todas las sentencias del texto.
    '''
    # Inicialización de variables
    terms = {}
    vocabulary = np.empty(0)

    # Diccionario con los documentos del corpus
    for k, v in enumerate(corpus):
        terms[k] = corpus[k].split(' ')
    
    # Vocabulario del corpus
    for key in terms.keys(): 
        vocabulary = np.concatenate((vocabulary, np.array(terms[key])), axis=None)
    
    return terms, np.unique(vocabulary)


In [4]:
# Impresión de documentos y vocabulario del corpus
terms, vocabulary = doc_voc(corpus)

for i in range(len(terms)):
    print(f'Documento {i}: {terms[i]}')

print(f'Array con el vocabulario del corpus: {doc_voc(corpus)[1]}')
print(f'Cantidad de términos que contine el vocabulario: {len(doc_voc(corpus)[1])}')

Documento 0: ['que', 'dia', 'es', 'hoy']
Documento 1: ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes']
Documento 2: ['martes', 'muchas', 'gracias']
Array con el vocabulario del corpus: ['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']
Cantidad de términos que contine el vocabulario: 9


### 2- OneHot encoding
Dada una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [5]:
# Función para contruir la matriz One Hot Encoding
def ohe(corpus):
    '''
    corpus: corpus que contiene todas las sentencias del texto.
    '''

    # Definición de variables
    terms, vocabulary = doc_voc(corpus)
    ohe = np.empty((0,len(vocabulary)), int)

    # Armado de la matriz
    for key in terms.keys():
        l = np.isin(vocabulary, terms[key])
        ohe = np.vstack((ohe, l))
    return ohe

In [6]:
OHE = ohe(corpus)
print(f'Matriz representativa OHE: \n {OHE}')

Matriz representativa OHE: 
 [[0 1 0 1 0 1 0 0 1]
 [1 1 1 1 0 1 1 0 0]
 [0 0 0 0 1 0 1 1 0]]


In [7]:
# Preparación de una mejor visualización del encoding OHE
idx = ['doc 1', 'doc 2', 'doc 3'] 
df_ohe = pd.DataFrame(ohe(corpus), columns=vocabulary, index=idx)
df_ohe

,de,dia,el,es,gracias,hoy,martes,muchas,que
doc 1,0,1,0,1,0,1,0,0,1
doc 2,1,1,1,1,0,1,1,0,0
doc 3,0,0,0,0,1,0,1,1,0


### 3- Vectores de frecuencia
Dada una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [8]:
# Función para contruir la matriz Frecuency Array

def fa(corpus):
    '''
    corpus: corpus que contiene todas las sentencias del texto.
    '''

    # Definición de variables
    terms, vocabulary = doc_voc(corpus)
    fa = np.empty((0,len(vocabulary)), int)

    # Armado de la matriz
    for key in terms.keys():
        r = Counter(terms[key])
        l = np.isin(vocabulary, terms[key]).astype(int)
        for i in range(len(l)): 
            if l[i] == 1:
                l[i] = r[vocabulary[i]]
            else:
                l[i] = 0
        fa = np.vstack((fa, l))
    return fa

In [9]:
FA = fa(corpus)
print(f'Matriz representativa Frecuency Array: \n {fa(corpus)}')  

Matriz representativa Frecuency Array: 
 [[0 1 0 1 0 1 0 0 1]
 [1 1 1 1 0 1 2 0 0]
 [0 0 0 0 1 0 1 1 0]]


In [10]:
# Preparación de una mejor visualización del encoding Frecuency Array 
df_fa = pd.DataFrame(fa(corpus), columns=vocabulary, index=idx)
df_fa

,de,dia,el,es,gracias,hoy,martes,muchas,que
doc 1,0,1,0,1,0,1,0,0,1
doc 2,1,1,1,1,0,1,2,0,0
doc 3,0,0,0,0,1,0,1,1,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [11]:
# Función para armar la matriz TF|IDF
def tf_idf(corpus):
    '''
    corpus: corpus que contiene todas las sentencias del texto.
    '''
    # Definición de variables
    terms, vocabulary = doc_voc(corpus)
    ohe = np.empty((0,len(vocabulary)), int)
    fa = np.empty((0,len(vocabulary)), int)

    # Armado de la matriz OHE
    for key in terms.keys():
        l = np.isin(vocabulary, terms[key])
        ohe = np.vstack((ohe, l))

    # Armado de la matriz Frecuency Array
    for key in terms.keys():
        r = Counter(terms[key])
        l = np.isin(vocabulary, terms[key]).astype(int)
        for i in range(len(l)): 
            if l[i] == 1:
                l[i] = r[vocabulary[i]]
            else:
                l[i] = 0
        fa = np.vstack((fa, l))
    
    # Factor IDF
    DF = np.sum(ohe, axis=0) # número total de documentos en la colección
    N = len(terms) # número de documentos en los que aparece un término del vocabulario
    mask = np.log10(N/DF) 

    # Matriz TF|IDF
    return fa*mask # la matriz Term Frecuency es la misma que se calculó en el punto anterior como "fa"


In [12]:
TF_IDF = tf_idf(corpus)
print(f'Matriz representativa TF|IDF: \n {TF_IDF}')  

Matriz representativa TF|IDF: 
 [[0.         0.17609126 0.         0.17609126 0.         0.17609126
  0.         0.         0.47712125]
 [0.47712125 0.17609126 0.47712125 0.17609126 0.         0.17609126
  0.35218252 0.         0.        ]
 [0.         0.         0.         0.         0.47712125 0.
  0.17609126 0.47712125 0.        ]]


In [13]:
# Preparación de una mejor visualización de la matriz TF|IDF
df_TF_IDF = pd.DataFrame(TF_IDF, columns=vocabulary, index=idx)
df_TF_IDF

,de,dia,el,es,gracias,hoy,martes,muchas,que
doc 1,0.000000,0.176091,0.000000,0.176091,0.000000,0.176091,0.000000,0.000000,0.477121
doc 2,0.477121,0.176091,0.477121,0.176091,0.000000,0.176091,0.352183,0.000000,0.000000
doc 3,0.000000,0.000000,0.000000,0.000000,0.477121,0.000000,0.176091,0.477121,0.000000


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [14]:
def simil_cos(corpus, d):
    '''
    Función que ordena los documentos del corpus por similitud coseno, a partir de un documento particular en el mismo.
    Devuelve un array cuya primer elemento es el documento a evaluar

    La similitud coseno se práctica sobre la matriz TF|IDF
    - corpus: corpus que contiene todas las sentencias del texto.
    - d: número de documento el cual se quiere evaluar.
    '''
    
    # Matriz TF|IDF del corpus
    x = tf_idf(corpus)
    # Referencias de los documentos a comparar
    r_doc = np.delete(np.arange(0, len(x), 1), d)
    
    # Calculo de similitud coseno documento a documento y se acumula en una lista su valor
    score = []
    for j in range(len(r_doc)):
          coseno = np.dot(x[d,:],x[j,:])/((np.linalg.norm(x[d,:]))*(np.linalg.norm(x[j,:])))
          score.append(coseno)
    
    # Indice ordenando de mayor a menor los cálculos de similitud
    idx = np.argsort(score)[::-1]
    # Las referencias de los documentos se ajustan al ordenamiento de los cálculos de similitud
    r_doc_sort = r_doc[idx]
    # Se agrega como primer elemento el documento evaluado
    r_doc_sort=np.insert(r_doc_sort, 0, d)

    # La función devuelve el corpus con los documentos ordenados según la similitud coseno y la referencia de los documentos
    return corpus[r_doc_sort], r_doc_sort
    

In [15]:
d = 2
corpus_similitud, referencias = simil_cos(corpus, d)

for i in range(len(referencias)):
    if i == 0:
        print(f'Documento de referencia {referencias[i]}: {corpus_similitud[i]}')
    else:
        print(f'Documento {referencias[i]}: {corpus_similitud[i]}')

Documento de referencia 2: martes muchas gracias
Documento 1: martes el dia de hoy es martes
Documento 0: que dia es hoy
